In [1]:
import numpy as np
import pandas as pd
import copy

import gensim
import sklearn
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.metrics import f1_score
from IPython.display import clear_output

In [24]:
df=pd.read_excel(r'mydatav2.xlsx')
dfclean=pd.read_excel('Clean Data.xlsx','Example')
dfall=pd.read_excel(r'C:\Users\Tran Anh Thuan\Desktop\Untitled Folder\dfall.xlsx',1)

In [25]:
dfapply=df[df['title']=='Nhà Giả Kim']

In [9]:
lstContentTest=dfapply['content'].to_list()

In [ ]:
dfPhanloai=pd.DataFrame(columns=['Word','Type'])
for j,c in enumerate(lstContentTest):
    for i,k in enumerate(dfall['Word'].to_list()):
        if k in c:
            dfPhanloai=dfPhanloai.append({'Word':c,'Type':dfall['Type2'].to_list()[i],'Stt':j},ignore_index=True)

# NLP

In [ ]:
tfidf_vect_ngram = TfidfVectorizer(ngram_range=(1, 2))
tfidf_vect_ngram.fit(df['content2'])
tfidf_vectorizer_vectors=tfidf_vect_ngram.fit_transform(df['content2'])
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(df['content2'])

In [ ]:
label=df['Label'][:400].to_list()

In [ ]:
filenametrainx = 'X_train.pkl'
filenametrainy = 'y_train.pkl'
filenamelog='logfile.txt'

X_train=copy.deepcopy(X_data_tfidf_ngram[:300])
y_train=copy.deepcopy(label[:300])

X_val=copy.deepcopy(X_data_tfidf_ngram[300:400])
y_val=copy.deepcopy(label[300:400])

"""
X_train = joblib.load(filenametrainx)
y_train=joblib.load(filenametrainy)
"""

In [ ]:
#Parameters
m=400
step=100
threshold=0.9
limit=25800
Cpara=10
gammapara=0.001
#Code
while True:
    if m>limit:
        break;

    strlog=str(X_train.shape)+"\n"
    clf=svm.SVC(kernel='rbf',C=Cpara,probability=True,gamma=gammapara)
    clf.fit(X_train, y_train)
    X_test=X_data_tfidf_ngram[m:m+step]
    result=clf.predict_proba(X_test)
    for i,prob in enumerate(result):
        if np.amax(prob)>=threshold:
            X_train=sparse.vstack((X_train,X_data_tfidf_ngram[m+i]))
            y_train.append(np.where(prob==np.amax(prob))[0][0]-1)

    val_predictions = clf.predict(X_val)
    strlog+=(str(m/100)+"\n")
    strlog+=(str(f1_score(val_predictions, y_val,average='macro'))+"\n")
    log=open(filenamelog,'w')
    log.write(strlog)
    log.close()
    clear_output(wait=True)
    pickle.dump(X_train, open(filenametrainx, 'wb'))
    pickle.dump(y_train, open(filenametrainy, 'wb'))
    print(strlog)
    m+=step

(15077, 168357)
197.0
0.7821268237934905

